In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
import os
from nltk import sent_tokenize
from nltk.corpus import stopwords


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def separate_into_sentences(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    sentences = sent_tokenize(text)
    return sentences
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])


In [ ]:
paths = ["anxiety_test.txt", "depression_test.txt", "bipolar_test.txt", "ocd_test.txt"]

In [ ]:
dfs = []
for i in range(len(paths)):
    path = paths[i]
    with open(path, 'r') as file:
      lines = file.readlines()
    patient_statements = [p.replace("Patient:", "").strip() for p in lines[::4]]  # Every even-indexed line
    therapist_responses = [p.replace("Therapist:", "").strip() for p in lines[2::4]]  # Every odd-indexed line
    df = pd.DataFrame({'Patient': patient_statements, 'Therapist': therapist_responses})
    dfs.append(df)
total_df = pd.concat(dfs, ignore_index=True)

In [ ]:

print(total_df)

                                              Patient  \
0   Lately, I've been feeling restless, tense, and...   
1   I find myself constantly anticipating the wors...   
2   I struggle with sleeping because my mind races...   
3   I tend to overthink even the smallest decision...   
4   I often experience physical symptoms like musc...   
5   I worry excessively about the well-being of my...   
6   I have this constant fear that something terri...   
7   I've been avoiding social situations because I...   
8   I feel constantly on edge, and it's challengin...   
9   I often experience a sense of impending danger...   
10  Lately, I've been feeling persistently sad, an...   
11  I constantly feel fatigued, even after a full ...   
12  I've been isolating myself from friends and fa...   
13  I find myself constantly questioning my self-w...   
14  I've been experiencing changes in my appetite,...   
15  I have trouble falling asleep at night, and ev...   
16  I'm experiencing a lack of 

In [ ]:
csv_file_path = './test_data.csv'


#some more cleaning

# Save the DataFrame to a CSV file
total_df.to_csv(csv_file_path, index=False)

print(f'DataFrame saved to {csv_file_path}')


DataFrame saved to ./test_data.csv


In [ ]:
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
from langchain.text_splitter import CharacterTextSplitter
import numpy as np
import pandas as pd
from langchain.vectorstores import Chroma
# Example text corpus
with open("test_data/dsm4.txt", 'r') as file:
    text_corpus = file.read()
text_splitter = RecursiveCharacterTextSplitter(
chunk_size=1000, chunk_overlap=50, separators=[" ", ",", "\n"]
)
texts = text_splitter.split_text(text_corpus)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'})
db = Chroma.from_texts(texts, embeddings)
retriever = db.as_retriever()

In [ ]:

retrieved_docs = retriever.invoke(
    "I've been having trouble sleeping and have no appetite?"
)
print(retrieved_docs[0].page_content)

In [ ]:
df = pd.read_csv("test_data.csv")

def f(x):
  return retriever.invoke(x)[0].page_content
df['context'] = df.apply(f, axis = 1)
len(df)